In [0]:
storage_account_name = "cadfstorage"
storage_account_key = "Y8cR8X6SmC7QLojvNRw1h95qe94HGA1GbGDFJxRvs06wf9DVfuy/NaVCEQ0nOO/VyFH7x8KaccIr+AStY6uuYw=="

# Configure Spark to use the storage account key
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_key)



In [0]:
# Assuming both files are in CSV format
accounts_df = spark.read.format("csv").option("header", "true").load("abfss://silver@cadfstorage.dfs.core.windows.net/part-00000-tid-7843773515758180079-bf13967a-365a-47ae-a354-7d2d491746ed-43-1-c000.csv")
customers_df = spark.read.format("csv").option("header", "true").load("abfss://silver@cadfstorage.dfs.core.windows.net/customers.csv")

accounts_df.show(5)
customers_df.show(5)

+-----------+----------+------------+--------+
|customer_id|account_id|account_type| balance|
+-----------+----------+------------+--------+
|         45|         1|     Savings| 1000.50|
|         50|       100|    Checking|10100.00|
|          3|        11|     Savings| 1100.75|
|         81|        12|    Checking| 2700.00|
|         29|        13|     Savings| 1300.25|
+-----------+----------+------------+--------+
only showing top 5 rows

+-----------+----------+---------+--------------+---------+-----+------+
|customer_id|first_name|last_name|       address|     city|state|   zip|
+-----------+----------+---------+--------------+---------+-----+------+
|          1|      John|      Doe|    123 Elm St|  Toronto|   ON|M4B1B3|
|          2|      Jane|    Smith| 456 Maple Ave|   Ottawa|   ON|K1A0B1|
|          3|   Michael|  Johnson|    789 Oak Dr| Montreal|   QC|H1A1A1|
|          4|     Emily|    Davis|   101 Pine Rd|  Calgary|   AB|T2A0A1|
|          5|     David|   Wilson|202 Bir

In [0]:
from pyspark.sql import functions as F

# Convert balance column to float if necessary
accounts_df = accounts_df.withColumn("balance", F.col("balance").cast("float"))

# Join customers and accounts DataFrames on customer_id and select specific columns to avoid ambiguity
joined_df = customers_df.alias("cust").join(accounts_df.alias("acct"), F.col("cust.customer_id") == F.col("acct.customer_id"), "inner") \
    .select(
        F.col("cust.customer_id").alias("customer_id"),
        "first_name",
        "last_name",
        "address",
        "city",
        "state",
        "zip",
        "account_type",
        "balance"
    )

# Aggregate to calculate the total balance for each customer
result_df = joined_df.groupBy(
    "customer_id",
    "first_name",
    "last_name",
    "address",
    "city",
    "state",
    "zip",
    "account_type"
).agg(
    F.sum("balance").alias("total_balance")
)

# Display the result
result_df.show()

+-----------+-----------+---------+---------------+---------------+-----+------+------------+-------------+
|customer_id| first_name|last_name|        address|           city|state|   zip|account_type|total_balance|
+-----------+-----------+---------+---------------+---------------+-----+------+------------+-------------+
|         86|     Olivia|   Gibson|    8585 Elm St|   New Liskeard|   ON|P0J0A1|     Savings|       400.25|
|         81|    Michael|    Owens| 8080 Willow Rd|        Mattawa|   ON|P0H0A1|    Checking|       2700.0|
|         26|    Abigail|   Parker| 2525 Poplar St|         Barrie|   ON|L4M0A1|    Checking|       6700.5|
|         20|        Mia|   Nelson|1919 Birch Blvd|         London|   ON|N6A0A1|    Checking|       6100.0|
|          5|      David|   Wilson| 202 Birch Blvd|      Vancouver|   BC|V5K0A1|    Checking|       1600.5|
|         53|      James|  Jenkins| 5252 Willow Rd|    Queensville|   ON|L0G0A1|     Savings|       300.25|
|         83|      David|   

In [0]:
# Write the sorted DataFrame to Azure Data Lake Storage in CSV format
result_df.write.format("csv").mode("overwrite").option("header", "true").save("abfss://silver@cadfstorage.dfs.core.windows.net/total_balance_per_customer.csv")


In [0]:
# Assuming `total_balance_df` is the DataFrame with the aggregated total balances
gold_container_path = "abfss://gold@cadfstorage.dfs.core.windows.net/total_balance_per_customer.csv"

result_df.write.format("csv").mode("overwrite").option("header", "true").save("abfss://gold@cadfstorage.dfs.core.windows.net/total_balance_per_customer.csv")

result_df.show()


+-----------+-----------+---------+---------------+---------------+-----+------+------------+-------------+
|customer_id| first_name|last_name|        address|           city|state|   zip|account_type|total_balance|
+-----------+-----------+---------+---------------+---------------+-----+------+------------+-------------+
|         86|     Olivia|   Gibson|    8585 Elm St|   New Liskeard|   ON|P0J0A1|     Savings|       400.25|
|         81|    Michael|    Owens| 8080 Willow Rd|        Mattawa|   ON|P0H0A1|    Checking|       2700.0|
|         26|    Abigail|   Parker| 2525 Poplar St|         Barrie|   ON|L4M0A1|    Checking|       6700.5|
|         20|        Mia|   Nelson|1919 Birch Blvd|         London|   ON|N6A0A1|    Checking|       6100.0|
|          5|      David|   Wilson| 202 Birch Blvd|      Vancouver|   BC|V5K0A1|    Checking|       1600.5|
|         53|      James|  Jenkins| 5252 Willow Rd|    Queensville|   ON|L0G0A1|     Savings|       300.25|
|         83|      David|   